In [1]:
# Run once before notebook 
import os
os.chdir("..")
os.getcwd()

'c:\\Users\\Rehan Ibrahim\\OneDrive - HEC Paris\\Desktop\\mckinsey-methane-hackathon'

In [2]:
from utils.dataloader import LoadData
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
import datetime
import rasterio
from sklearn.model_selection import GroupShuffleSplit

In [4]:
meta_df = pd.read_csv('data/dataset/train_data/metadata.csv')

<IPython.core.display.Javascript object>

In [5]:
meta_df[['lat','lon']].value_counts()

lat        lon       
29.631951   35.952379    21
32.713854   44.609398    19
33.990812   39.641866    18
28.510000   77.442400    17
36.596520   38.321405    15
                         ..
21.039986  -77.824694     1
24.907500   67.023000     1
23.763333   86.396667     1
23.740000   90.595000     1
68.570113   25.563059     1
Length: 101, dtype: int64

In [6]:
meta_df.head()

,date,id_coord,plume,set,lat,lon,coord_x,coord_y,path
0,20230223,id_6675,yes,train,31.528750,74.330625,24,47,images/plume/20230223_methane_mixing_ratio_id_...
1,20230103,id_2542,yes,train,35.538000,112.524000,42,37,images/plume/20230103_methane_mixing_ratio_id_...
2,20230301,id_6546,yes,train,21.060000,84.936667,58,15,images/plume/20230301_methane_mixing_ratio_id_...
3,20230225,id_6084,yes,train,26.756667,80.973333,28,62,images/plume/20230225_methane_mixing_ratio_id_...
4,20230105,id_2012,yes,train,34.800000,40.770000,59,44,images/plume/20230105_methane_mixing_ratio_id_...


In [7]:
meta_df['path'] = 'data/dataset/train_data/' + meta_df['path'].astype(str) 
meta_df['path'] =  meta_df['path'].astype(str) +'.tif'

In [8]:
meta_df

,date,id_coord,plume,set,lat,lon,coord_x,coord_y,path
0,20230223,id_6675,yes,train,31.528750,74.330625,24,47,data/dataset/train_data/images/plume/20230223_...
1,20230103,id_2542,yes,train,35.538000,112.524000,42,37,data/dataset/train_data/images/plume/20230103_...
2,20230301,id_6546,yes,train,21.060000,84.936667,58,15,data/dataset/train_data/images/plume/20230301_...
3,20230225,id_6084,yes,train,26.756667,80.973333,28,62,data/dataset/train_data/images/plume/20230225_...
4,20230105,id_2012,yes,train,34.800000,40.770000,59,44,data/dataset/train_data/images/plume/20230105_...
...,...,...,...,...,...,...,...,...,...
425,20230302,id_6658,no,train,30.965619,34.541283,39,36,data/dataset/train_data/images/no_plume/202303...
426,20230218,id_4690,no,train,35.950275,40.267652,29,28,data/dataset/train_data/images/no_plume/202302...
427,20230213,id_2519,no,train,49.963801,6.016938,23,10,data/dataset/train_data/images/no_plume/202302...
428,20230213,id_5510,no,train,32.713854,44.609398,55,54,data/dataset/train_data/images/no_plume/202302...


In [9]:
meta_df['plume'] = meta_df['plume'].map({'yes': 1, 'no': 0})

In [10]:
meta_df

,date,id_coord,plume,set,lat,lon,coord_x,coord_y,path
0,20230223,id_6675,1,train,31.528750,74.330625,24,47,data/dataset/train_data/images/plume/20230223_...
1,20230103,id_2542,1,train,35.538000,112.524000,42,37,data/dataset/train_data/images/plume/20230103_...
2,20230301,id_6546,1,train,21.060000,84.936667,58,15,data/dataset/train_data/images/plume/20230301_...
3,20230225,id_6084,1,train,26.756667,80.973333,28,62,data/dataset/train_data/images/plume/20230225_...
4,20230105,id_2012,1,train,34.800000,40.770000,59,44,data/dataset/train_data/images/plume/20230105_...
...,...,...,...,...,...,...,...,...,...
425,20230302,id_6658,0,train,30.965619,34.541283,39,36,data/dataset/train_data/images/no_plume/202303...
426,20230218,id_4690,0,train,35.950275,40.267652,29,28,data/dataset/train_data/images/no_plume/202302...
427,20230213,id_2519,0,train,49.963801,6.016938,23,10,data/dataset/train_data/images/no_plume/202302...
428,20230213,id_5510,0,train,32.713854,44.609398,55,54,data/dataset/train_data/images/no_plume/202302...


In [11]:
with rasterio.open('data/dataset/train_data/images/plume/20230223_methane_mixing_ratio_id_6675.tif') as src:
    img = src.read(1)
ep.plot_bands(img)

c:\Users\Rehan Ibrahim\AppData\Local\Programs\Python\Python310\lib\site-packages\rasterio\__init__.py:320: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


NameError: name 'ep' is not defined

In [12]:
image_data = []
plume_labels = []

# Loop through the metadata and load images
for index, row in meta_df.iterrows():
    image_path = row['path']
    plume_label = row['plume']

    # Read the TIFF image using rasterio
    try:
        with rasterio.open(image_path) as src:
            image = src.read(1)  # Assuming single-band image, adjust if necessary
            # You may want to resize or preprocess the image here if necessary

        # Append the image data and plume label to their respective lists
        image_data.append(image)
        plume_labels.append(plume_label)
    except Exception as e:
        print(f"Error loading image at {image_path}: {e}")

# Convert the lists into NumPy arrays
image_data = np.array(image_data)
plume_labels = np.array(plume_labels)

c:\Users\Rehan Ibrahim\AppData\Local\Programs\Python\Python310\lib\site-packages\rasterio\__init__.py:320: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
plume_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
image_data

array([[[15373, 23745, 23745, ...,     0,     0,     0],
        [21083, 22849, 22849, ...,     0,     0,     0],
        [21083, 22849, 22849, ...,     0,  3348,  3348],
        ...,
        [49287, 49287, 43637, ..., 22215, 29933, 29933],
        [49287, 43637, 43637, ..., 13711, 14626, 14626],
        [49834, 48903, 48903, ..., 13711, 14626, 14626]],

       [[    0,     0,     0, ...,     0,     0,     0],
        [ 6386, 11516,  1537, ...,     0,     0,     0],
        [ 6386, 11516,  1537, ...,     0,     0,     0],
        ...,
        [ 3878,  6976,  6548, ...,     0,     0,     0],
        [ 3878,  6976,  6548, ...,     0,     0,     0],
        [ 4225,     0,  4100, ...,     0,     0,     0]],

       [[14118,  9487,  9487, ..., 25601, 22138, 22138],
        [16914,  5666,  5666, ..., 30095, 26363, 26363],
        [16914,  5666,  5666, ..., 30095, 26363, 26363],
        ...,
        [14918, 14918, 16676, ..., 18546,     0,     0],
        [14918, 14918, 16676, ..., 18546,    

In [15]:
min_val = image_data.min()
max_val = image_data.max()

# Normalize the image data
normalized_image_data = (image_data - min_val) / (max_val - min_val)
normalized_image_data

array([[[0.23457694, 0.36232547, 0.36232547, ..., 0.        ,
         0.        , 0.        ],
        [0.32170596, 0.34865339, 0.34865339, ..., 0.        ,
         0.        , 0.        ],
        [0.32170596, 0.34865339, 0.34865339, ..., 0.        ,
         0.05108721, 0.05108721],
        ...,
        [0.75207141, 0.75207141, 0.66585794, ..., 0.33897917,
         0.4567483 , 0.4567483 ],
        [0.75207141, 0.66585794, 0.66585794, ..., 0.20921645,
         0.22317845, 0.22317845],
        [0.7604181 , 0.74621195, 0.74621195, ..., 0.20921645,
         0.22317845, 0.22317845]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.09744411, 0.1757229 , 0.02345312, ..., 0.        ,
         0.        , 0.        ],
        [0.09744411, 0.1757229 , 0.02345312, ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.05917449, 0.10644694, 0.09991608, ..., 0.        ,
         0.        , 0.        ],
        [0.0

In [16]:
X = normalized_image_data
y = plume_labels


#X = [np.transpose(image, (1, 2, 0)) for image in X]

#X = np.array(X)
np.random.seed(101)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42,shuffle=True)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

<IPython.core.display.Javascript object>

X_train shape: (258, 64, 64)
X_test shape: (172, 64, 64)


In [17]:
y_train

array([1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1])

In [25]:

data = LoadData(metadata_path="data/dataset/train_data/metadata.csv",
                image_data_path="data/dataset/train_data/")

X_train_aug, X_test, y_train_aug, y_test = data.prep_data(normalize=True, augment=True)

c:\Users\Rehan Ibrahim\AppData\Local\Programs\Python\Python310\lib\site-packages\rasterio\__init__.py:320: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [22]:
# Set the random seed for reproducibility
tf.random.set_seed(1234)

# Define the CNN model
model = tf.keras.Sequential([
    # Convolutional layers
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Flatten the output for fully connected layers
    layers.Flatten(),
    
    # Fully connected layers
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Use 'sigmoid' for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use 'binary_crossentropy' for binary classification
              metrics=['accuracy'])

# Print the model summary to view the architecture
model.summary()

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5,           
    restore_best_weights=True)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 6, 6, 128)        

In [5]:
batch_size = 32
epochs = 100
model.fit(X_train_aug, y_train_aug, 
          epochs=epochs, 
          batch_size=batch_size, 
          validation_split=0.2,
          callbacks=[early_stopping, tensorboard_callback])

Epoch 1/100
260/260 [==============================] - 8s 29ms/step - loss: 0.5214 - accuracy: 0.7314 - val_loss: 0.3727 - val_accuracy: 0.8308
Epoch 2/100
260/260 [==============================] - 7s 29ms/step - loss: 0.3289 - accuracy: 0.8659 - val_loss: 0.2224 - val_accuracy: 0.9115
Epoch 3/100
260/260 [==============================] - 8s 29ms/step - loss: 0.2100 - accuracy: 0.9232 - val_loss: 0.1611 - val_accuracy: 0.9298
Epoch 4/100
260/260 [==============================] - 7s 29ms/step - loss: 0.1134 - accuracy: 0.9597 - val_loss: 0.1170 - val_accuracy: 0.9548
Epoch 5/100
260/260 [==============================] - 8s 29ms/step - loss: 0.0771 - accuracy: 0.9714 - val_loss: 0.1206 - val_accuracy: 0.9543
Epoch 6/100
260/260 [==============================] - 8s 30ms/step - loss: 0.0536 - accuracy: 0.9830 - val_loss: 0.0560 - val_accuracy: 0.9793
Epoch 7/100
260/260 [==============================] - 8s 29ms/step - loss: 0.0531 - accuracy: 0.9829 - val_loss: 0.0487 - val_accuracy:

In [10]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.


# Concatenate External features

In [26]:
data.meta_df

,date,id_coord,plume,set,lat,lon,coord_x,coord_y,path
0,2023-02-23,id_6675,yes,train,31.528750,74.330625,24,47,images/plume/20230223_methane_mixing_ratio_id_...
1,2023-01-03,id_2542,yes,train,35.538000,112.524000,42,37,images/plume/20230103_methane_mixing_ratio_id_...
2,2023-03-01,id_6546,yes,train,21.060000,84.936667,58,15,images/plume/20230301_methane_mixing_ratio_id_...
3,2023-02-25,id_6084,yes,train,26.756667,80.973333,28,62,images/plume/20230225_methane_mixing_ratio_id_...
4,2023-01-05,id_2012,yes,train,34.800000,40.770000,59,44,images/plume/20230105_methane_mixing_ratio_id_...
...,...,...,...,...,...,...,...,...,...
425,2023-03-02,id_6658,no,train,30.965619,34.541283,39,36,images/no_plume/20230302_methane_mixing_ratio_...
426,2023-02-18,id_4690,no,train,35.950275,40.267652,29,28,images/no_plume/20230218_methane_mixing_ratio_...
427,2023-02-13,id_2519,no,train,49.963801,6.016938,23,10,images/no_plume/20230213_methane_mixing_ratio_...
428,2023-02-13,id_5510,no,train,32.713854,44.609398,55,54,images/no_plume/20230213_methane_mixing_ratio_...


In [27]:
def encode_dates(df, date_col="date"):
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['day'] = df[date_col].dt.day
    return df.drop("date", axis=1)

meta_df = encode_dates(data.meta_df.copy())
external_data = meta_df[["lat", "lon", "coord_x", "coord_y", "year", "month", "day"]]
external_data.head()

,lat,lon,coord_x,coord_y,year,month,day
0,31.528750,74.330625,24,47,2023,2,23
1,35.538000,112.524000,42,37,2023,1,3
2,21.060000,84.936667,58,15,2023,3,1
3,26.756667,80.973333,28,62,2023,2,25
4,34.800000,40.770000,59,44,2023,1,5


In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define the CNN model
model = tf.keras.Sequential([
    # Convolutional layers
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    # Flatten the output for fully connected layers
    layers.Flatten(),
])

# Metadata input layer
metadata_input = tf.keras.layers.Input(shape=(external_data.shape[1]))

# Get the output of the CNN model
cnn_output = model.output

# Concatenate the flattened output with the metadata input
concatenated = tf.keras.layers.concatenate([cnn_output, metadata_input])

# Add fully connected layers after concatenation
x = layers.Dense(128, activation='relu')(concatenated)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

# Create the combined model
combined_model = tf.keras.Model(inputs=[model.input, metadata_input], outputs=output)

# Compile the model
combined_model.compile(
    loss='binary_crossentropy',  
    optimizer='adam',  
    metrics=['accuracy'],  
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 6, 6, 128)        

In [29]:
# Train the model
combined_model.fit(
    [X_train, external_data],  
    y_train,  
    epochs=10,  
    batch_size=32,  
    validation_split=0.2,  
)

Epoch 1/10
7/7 [==============================] - 2s 108ms/step - loss: 17.7812 - accuracy: 0.4660 - val_loss: 10.2391 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 0s 71ms/step - loss: 10.6059 - accuracy: 0.4903 - val_loss: 6.7134 - val_accuracy: 0.5000
Epoch 3/10
7/7 [==============================] - 0s 72ms/step - loss: 10.0554 - accuracy: 0.4709 - val_loss: 2.4589 - val_accuracy: 0.5000
Epoch 4/10
7/7 [==============================] - 1s 74ms/step - loss: 8.0796 - accuracy: 0.4757 - val_loss: 2.4245 - val_accuracy: 0.5000
Epoch 5/10
7/7 [==============================] - 0s 71ms/step - loss: 5.6305 - accuracy: 0.4709 - val_loss: 0.6390 - val_accuracy: 0.6923
Epoch 6/10
7/7 [==============================] - 1s 75ms/step - loss: 4.3026 - accuracy: 0.5583 - val_loss: 0.8122 - val_accuracy: 0.3846
Epoch 7/10
7/7 [==============================] - 1s 72ms/step - loss: 2.8962 - accuracy: 0.5340 - val_loss: 1.7454 - val_accuracy: 0.5000
Epoch 8/10
7/7 [======

In [30]:
# Make predictions on the test dataset
predictions = model.predict(X_test)
predictions

3/3 [==============================] - 0s 16ms/step


array([[4.68729198e-01, 0.00000000e+00, 2.86045074e-02, ...,
        2.62588680e-01, 2.08794102e-01, 0.00000000e+00],
       [7.48590350e-01, 0.00000000e+00, 5.52959293e-02, ...,
        3.76618624e-01, 1.77203745e-01, 0.00000000e+00],
       [5.79662204e-01, 0.00000000e+00, 2.95998156e-02, ...,
        2.83273876e-01, 1.04130976e-01, 0.00000000e+00],
       ...,
       [1.89571306e-02, 0.00000000e+00, 1.00764260e-03, ...,
        2.88628995e-01, 1.17505126e-01, 0.00000000e+00],
       [1.48526883e+00, 0.00000000e+00, 7.41923526e-02, ...,
        3.52907896e-01, 9.56372321e-02, 0.00000000e+00],
       [3.13669652e-01, 0.00000000e+00, 6.02501445e-03, ...,
        5.27367711e-01, 2.22998023e-01, 0.00000000e+00]], dtype=float32)

In [31]:
# Apply threshold to obtain binary predictions (0 or 1)
threshold = 0.5
binary_predictions = (predictions >= threshold).astype(int)

In [33]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
accuracy = accuracy_score(y_test, binary_predictions)
print(f"Accuracy: {accuracy*100:.2f}%")

ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets

In [4]:
# Define the CNN model
model = tf.keras.Sequential([
    # Convolutional layers
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    # Flatten the output for fully connected layers
    layers.Flatten(),
])



input_1d = tf.keras.layers.Input(shape=(external_data.shape[1]))  # Adjust the shape as needed

# Concatenate the flattened output and the 1-dimensional input
X_concat = tf.keras.layers.concatenate([model.output, input_1d])

# Add fully connected layers after concatenation
concat_model = tf.keras.Sequential([
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')])

# Create the model
combined_model = tf.keras.Model(inputs=[model.input, input_1d])


<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_1')>